$$
PE(\text{position}, 2i) = \sin\bigg( \frac{ \text{position} }{10000^\frac{2i}{d_{model}}} \bigg)
$$

$$
PE(\text{position}, 2i+1) = \cos\bigg( \frac{ \text{position} }{10000^\frac{2i}{d_{model}}} \bigg)
$$

We can rewrite these as

$$
PE(\text{position}, i) = \sin\bigg( \frac{ \text{position} }{10000^\frac{i}{d_{model}}} \bigg) \text{ when i is even}
$$

$$
PE(\text{position}, i) = \cos\bigg( \frac{ \text{position} }{10000^\frac{i-1}{d_{model}}} \bigg) \text{ when i is odd}
$$

In [ ]:
""" From the postional Encoding formula,
i = dimention index
d_model = Embedding_length
pos = position of word in sequence

Why we used sin & cos in the positional encoding function?
>> Periodicity, these are periodic function those are reapted after a certain time.
>> Contrained values, values between position & negative values, which helps to capture more relevent information.
>> Easy to extrapolate for long sequences"""

' From the postional Encoding formula,\ni = dimention index\nd_model = Embedding_length\npos = position of word in sequence\n\nWhy we used sin & cos in the positional encoding function?\n>> Periodicity, these are periodic function those are reapted after a certain time.\n>> Contrained values, values between position & negative values, which helps to capture more relevent information.\n>> Easy to extrapolate for long sequences'

In [ ]:
import torch
import torch.nn as nn

In [ ]:
max_sequence_length = 10  ## length of the input sequence(sentence length)
d_model = 6 #dimention of the embedding vector of each word

In [ ]:
## lets start coding for the denominator of the formula when i is even, mean using sin.
 ##as I even, so started from Zero(0), till limit d_model =6, and 2 is the interval(that we can get all positive values.)
even_i = torch.arange(0,d_model, 2).float()
even_i

tensor([0., 2., 4.])

In [ ]:
## denominator of the even
even_denominator = torch.pow(10000, even_i / d_model) #pow is to the power
even_denominator

tensor([  1.0000,  21.5443, 464.1590])

In [ ]:
# for the odd
odd_i = torch.arange(1, d_model, 2 ) ## odd so, started from 1 that we can odd values
odd_i

tensor([1, 3, 5])

In [ ]:
## denominator of the odd
odd_denominator = torch.pow(10000, (odd_i - 1)/d_model) # pow is to the power
odd_denominator

tensor([  1.0000,  21.5443, 464.1590])

In [ ]:
""" as the even_denominator and odd_denominator provides are same values
and actually the same thing. So, we can consider as one"""

denominator = even_denominator

In [ ]:
## now we define every single position by defining every sequence [ Numerator-part ]
position = torch.arange(max_sequence_length, dtype = torch.float).reshape(max_sequence_length, 1) ## reshape into two dimentional matrix (one for everyword)
position

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])

In [ ]:
## now lets impliment the final equation
even_PE = torch.sin(position / denominator) # for even, sin
odd_PE = torch.cos(position / denominator) # for off, cos

In [ ]:
even_PE

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.8415,  0.0464,  0.0022],
        [ 0.9093,  0.0927,  0.0043],
        [ 0.1411,  0.1388,  0.0065],
        [-0.7568,  0.1846,  0.0086],
        [-0.9589,  0.2300,  0.0108],
        [-0.2794,  0.2749,  0.0129],
        [ 0.6570,  0.3192,  0.0151],
        [ 0.9894,  0.3629,  0.0172],
        [ 0.4121,  0.4057,  0.0194]])

In [ ]:
even_PE.shape

torch.Size([10, 3])

In [ ]:
odd_PE

tensor([[ 1.0000,  1.0000,  1.0000],
        [ 0.5403,  0.9989,  1.0000],
        [-0.4161,  0.9957,  1.0000],
        [-0.9900,  0.9903,  1.0000],
        [-0.6536,  0.9828,  1.0000],
        [ 0.2837,  0.9732,  0.9999],
        [ 0.9602,  0.9615,  0.9999],
        [ 0.7539,  0.9477,  0.9999],
        [-0.1455,  0.9318,  0.9999],
        [-0.9111,  0.9140,  0.9998]])

In [ ]:
odd_PE.shape

torch.Size([10, 3])

In [ ]:
""""now we have odd and even position separately, now we have to combine them together,
as we need to adjust the position along with index of the words like index 0,1,2,4,
so we will stack the even and odd positional encoding each other"""
stacked = torch.stack([even_PE, odd_PE], dim = 2)
stacked.shape

torch.Size([10, 3, 2])

In [ ]:
""" after stack on odd and even PE on each other, we flatten it
So for the first word [ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000]
for second word [ 0.8415,  0.5403,  0.0464,  0.9989,  0.0022,  1.0000], this
one is the embedding and so on."""
PE = torch.flatten(stacked, start_dim = 1, end_dim =2)
PE

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0464,  0.9989,  0.0022,  1.0000],
        [ 0.9093, -0.4161,  0.0927,  0.9957,  0.0043,  1.0000],
        [ 0.1411, -0.9900,  0.1388,  0.9903,  0.0065,  1.0000],
        [-0.7568, -0.6536,  0.1846,  0.9828,  0.0086,  1.0000],
        [-0.9589,  0.2837,  0.2300,  0.9732,  0.0108,  0.9999],
        [-0.2794,  0.9602,  0.2749,  0.9615,  0.0129,  0.9999],
        [ 0.6570,  0.7539,  0.3192,  0.9477,  0.0151,  0.9999],
        [ 0.9894, -0.1455,  0.3629,  0.9318,  0.0172,  0.9999],
        [ 0.4121, -0.9111,  0.4057,  0.9140,  0.0194,  0.9998]])

Create a class for the positional endodering, its what we have done so far!

In [ ]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1)
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

In [ ]:
## apply and use the positional encoding function
pe = PositionalEncoding(d_model=6, max_sequence_length=10)
pe.forward()

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0464,  0.9989,  0.0022,  1.0000],
        [ 0.9093, -0.4161,  0.0927,  0.9957,  0.0043,  1.0000],
        [ 0.1411, -0.9900,  0.1388,  0.9903,  0.0065,  1.0000],
        [-0.7568, -0.6536,  0.1846,  0.9828,  0.0086,  1.0000],
        [-0.9589,  0.2837,  0.2300,  0.9732,  0.0108,  0.9999],
        [-0.2794,  0.9602,  0.2749,  0.9615,  0.0129,  0.9999],
        [ 0.6570,  0.7539,  0.3192,  0.9477,  0.0151,  0.9999],
        [ 0.9894, -0.1455,  0.3629,  0.9318,  0.0172,  0.9999],
        [ 0.4121, -0.9111,  0.4057,  0.9140,  0.0194,  0.9998]])